In [1]:
# Defining random seeds to enable reproducibility
from numpy.random import seed
seed(1)

import tensorflow as tf
tf.random.set_seed(1)
 
import random
random.seed(1)

import pickle
import numpy as np
from tensorflow.keras import models
from tensorflow.python.client import device_lib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

 
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


def load_patras_dataset(variant='keyframe'):
    """
    Loads the datasets encoded in .pkl files and returns its decoded form.

    Parameters
    ----------
    variant : {'keyframe', 'multiframe'}
        A string indicating the dataset version to load.
    
    Returns
    -------
    list
        A list of n-dimensional arrays representing the subjects samples that will be used to \\
        test the drunkenness classification model.
    ndarray
        A n-dimensional array representing the test set labels.
    """

    print("Loading Sober-Drunk Face Dataset, from Patras University")
    
    # Defining the sample and label sets filenames according to the variant parameter
    if variant == 'keyframe':
        sets = [
            "Insert the keyframe-based unbalanced x_test-set.pkl file path here",
            "Insert the keyframe-based unbalanced y_test-set.pkl file path here"
        ]
    else:
        sets = [
            "Insert the multiframe-based unbalanced x_test-set.pkl file path here",
            "Insert the multiframe-based unbalanced y_test-set.pkl file path here"
        ]
    
    # Defining an empty list for storing the decoded datasets
    loaded_datasets = []
 
    # Iterating over the dataset files
    for set_ in sets:
        # Opening the .pkl file in read mode
        with open(set_, 'rb') as file:
            # Appending the decoded dataset to the datasets list
            loaded_datasets.append(pickle.load(file))
    
    # Unpacking the datasets list into individual subsets
    x_test, y_test = loaded_datasets

    # Converting the label list to the n-dimensional array format
    y_test = np.array(y_test)
 
    # Printing the dataset length
    print("\nTest set samples: {0}".format(len(x_test)))
    
    # Returning the test set samples and its respective labels
    return x_test, y_test
        

def min_max_norm(dataset):
    """
    Normalizes the keyframes according to the minimum-maximum norm, \\
    such that pixel values ranges from 0 to 1.

    Parameters
    ----------
    dataset : list
        A list of n-dimensional arrays representing the subjects keyframes.

    Returns
    -------
    ndarray
        A n-dimensional array representing keyframes with pixel values ranging from 0 to 1.
    """

    # Converting the dataset type from list to n-dimensional array
    dataset = np.asarray(dataset, dtype="int16")

    # Finding the keyframes minimum and maximum values
    x_min = dataset.min(axis=(1, 2), keepdims=True)
    x_max = dataset.max(axis=(1, 2), keepdims=True)

    # Applying the minimum-maximum norm to each keyframe
    norm_dataset = (dataset - x_min) / (x_max - x_min)

    # Printing the minimum and maximum values from a given sample for sanity check
    print("\nMinMax normalization")
    print("dataset shape: ", norm_dataset.shape)
    print("min: ", norm_dataset[0].min())
    print("max: ", norm_dataset[0].max())

    # Returning the normalized dataset
    return norm_dataset


def print_classification_report(y_test, y_hat, dataset_variant='keyframe'):
    """
    Prints the final model classification performance assessment and the confusion \\
    matrix comparing the predicted classes in relation to the real ones.

    Parameters
    ----------
    y_test : ndarray
        A n-dimensional array representing the test set true labels.
    y_hat : ndarray
        A n-dimensional array representing the test set labels predicted by the final \\
        model.
    dataset_variant : {'keyframe', 'multiframe'}
        A string indicating the dataset version in which the classification model will \\ 
        be evaluated.
    """

    # Printing the classification performance report
    report = classification_report(y_test, y_hat, target_names=['sober', 'drunk'])
    print(report)
 
    print("\nConfusion Matrix: ")

    # Printing the confusion matrix
    matrix = confusion_matrix(y_test, y_hat)
    print(matrix)
 
    tn, fp, fn, tp = matrix.ravel()
    print("\nTrue Negatives: ", tn)
    print("False Positives: ", fp)
    print("False Negatives: ", fn)
    print("True Positives: ", tp)


    # Flattening the n-dimensional label arrays
    true_classes = y_test.ravel()
    predicted_classes = y_hat.ravel()    
    
    # Printing the true and predicted labels of each subject
    if dataset_variant == 'keyframe':
        print("\nsubject 37 true classes: \t{0}".format(true_classes[0:4]))
        print("subject 37 predicted classes: \t{0}".format(predicted_classes[0:4]))

        print("\nsubject 38 true classes: \t{0}".format(true_classes[4:8]))
        print("subject 38 predicted classes: \t{0}".format(predicted_classes[4:8]))

        print("\nsubject 39 true classes: \t{0}".format(true_classes[8:12]))
        print("subject 39 predicted classes: \t{0}".format(predicted_classes[8:12]))

        print("\nsubject 40 true classes: \t{0}".format(true_classes[12:16]))
        print("subject 40 predicted classes: \t{0}".format(predicted_classes[12:16]))

        print("\nsubject 41 true classes: \t{0}".format(true_classes[16:20]))
        print("subject 41 predicted classes: \t{0}".format(predicted_classes[16:20]))
    else:
        print("\nsubject 37 true classes: \n{0}".format(true_classes[0:200]))
        print("\nsubject 37 predicted classes: \n{0}".format(predicted_classes[0:200]))

        print("\nsubject 38 true classes: \n{0}".format(true_classes[200:400]))
        print("\nsubject 38 predicted classes: \n{0}".format(predicted_classes[200:400]))

        print("\nsubject 39 true classes: \n{0}".format(true_classes[400:600]))
        print("\nsubject 39 predicted classes: \n{0}".format(predicted_classes[400:600]))

        print("\nsubject 40 true classes: \n{0}".format(true_classes[600:800]))
        print("\nsubject 40 predicted classes: \n{0}".format(predicted_classes[600:800]))

        print("\nsubject 41 true classes: \n{0}".format(true_classes[800:1000]))
        print("\nsubject 41 predicted classes: \n{0}".format(predicted_classes[800:1000]))


print("\n\nClassification performance assessment:\n")

# Loading the test dataset
x_test, y_test = load_patras_dataset()

# Applying the min-max normalization
x_test = min_max_norm(x_test)

# Reshaping datsets to the tensor format (channel last)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
          
print("\nLoading the final drunkenness classification model")

# Loading the final drunkenness classification model
model = models.load_model('sober-drunk_final-model_ft_fold-3.h5')

# Printing the final model summary
model.summary()

# Obtaining the test samples class probabilities   
y_prob = model.predict(x_test)

# Obtaining the binary label from the model output probabilities
y_hat = (y_prob > 0.5).astype(int)

print("\nClassification performance on the keyframe dataset: ")

# Printing the final model classification performance assessment on the keyframe dataset
print_classification_report(y_test, y_hat)

print("\n==========================================================================================\n")

print("Generalization assessment:\n")

# Loading the test dataset multiframe version
x_test, y_test = load_patras_dataset('multiframe')

# Applying the min-max normalization
x_test = min_max_norm(x_test)

# Reshaping datsets to the tensor format (channel last)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)

# Obtaining the test samples class probabilities   
y_prob = model.predict(x_test)

# Obtaining the binary label from the model output probabilities
y_hat = (y_prob > 0.5).astype(int)

print("\nClassification performance on the multiframe dataset: ")

# Printing the final model classification performance assessment on the multiframe dataset
print_classification_report(y_test, y_hat, 'multiframe')


2022-08-08 15:10:38.564445: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-08 15:10:38.628326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 15:10:38.792924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 15:10:38.793784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17185065197945081162
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16152002560
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2271303286835939952
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]
Num GPUs Available:  1


Classification performance assessment:

Loading Sober-Drunk Face Dataset, from Patras University

Test set samples: 20

MinMax normalization
dataset shape:  (20, 128, 160, 3)
min:  0.0
max:  1.0

Loading the final drunkenness classification model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 160, 64)      36928     
_________

2022-08-08 15:10:44.050406: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-08 15:10:45.101403: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005



Classification performance on the keyframe dataset: 
              precision    recall  f1-score   support

       sober       1.00      0.80      0.89         5
       drunk       0.94      1.00      0.97        15

    accuracy                           0.95        20
   macro avg       0.97      0.90      0.93        20
weighted avg       0.95      0.95      0.95        20


AUC ROC: 
0.8800000000000001

Confusion Matrix: 
[[ 4  1]
 [ 0 15]]

True Negatives:  4
False Positives:  1
False Negatives:  0
True Positives:  15

subject 37 true classes: 	[0 1 1 1]
subject 37 predicted classes: 	[0 1 1 1]

subject 38 true classes: 	[0 1 1 1]
subject 38 predicted classes: 	[0 1 1 1]

subject 39 true classes: 	[0 1 1 1]
subject 39 predicted classes: 	[0 1 1 1]

subject 40 true classes: 	[0 1 1 1]
subject 40 predicted classes: 	[0 1 1 1]

subject 41 true classes: 	[0 1 1 1]
subject 41 predicted classes: 	[1 1 1 1]


Generalization assessment:

Loading Sober-Drunk Face Dataset, from Patras Univ